# Zagadnienie przydziału - metoda węgierska
<b>Grupa: </b> 4a, czwartek 14:30 – 16:00 

<b>Data wykonania: </b> 29.04.2021

<b> Skład zespołu: </b>

-Zuzanna Zielińska

-Zofia Lenarczyk

-Maciej Kucharski

-Łukasz Rams

### Importy potrzebnych bibliotek

In [ ]:
import numpy as np
from typing import Tuple, Optional, Union, Tuple
from collections import Counter
from itertools import product

## Krok 1 - Redukcja macierzy

In [ ]:
import numpy as np
from typing import Tuple, Optional

def reduce_matrix(matrix: np.ndarray) -> Optional[Tuple[np.ndarray, int]]:

    """
    Funkcja realizuje redukcję macierzy kwadratowej.
    W każdym wierszu wyszukiwany jest element najmniejszy, który następnie jest dodawany do kosztu redukcji oraz
    odejmowany (redukowany) od wszystkich elementów w wierszu.
    Analogiczne działanie dla kolumn macierzy.

    :param matrix: macierz wejściowa
    :return: zredukowana macierz, koszt redukcji
    """

    # Sprawdzenie czy macierz jest kwadratowa:
    if matrix.shape[0] != matrix.shape[1]:
        print('Niepoprawne wymiary macierzy wejściowej')
        return None

    reduction_cost: int = 0  # Koszt redukcji
    reduced_rows_matrix: np.ndarray = np.zeros(matrix.shape)  # Macierz z zredukowanymi wierszami
    reduced_matrix: np.ndarray = np.zeros(matrix.shape)  # Zredukowana macierz

    # Redukcja wierszy
    for i in range(matrix.shape[0]):
        minimum = matrix[i].min()
        reduction_cost += minimum
        row_reduced = [x - minimum for x in matrix[i]]
        reduced_rows_matrix[i] = row_reduced

    print(f'Macierz ze zredukowanymi wierszami:\n {reduced_rows_matrix}')
    print(f'Koszt redukcji wierszy: {reduction_cost}')

    # Redukcja kolumn
    for i in range(matrix.shape[0]):
        minimum = reduced_rows_matrix[:, i].min()
        reduction_cost += minimum
        col_reduced = [x - minimum for x in reduced_rows_matrix[:, i]]
        reduced_matrix[:, i] = col_reduced

    print(f'Zredukowana macierz:\n {reduced_matrix}')
    print(f'Całkowity koszt redukcji macierzy: {reduction_cost}')

    return reduced_matrix, reduction_cost

## Krok 2 - Poszukiwanie kompletnego przydziału

In [ ]:
def set_zeros(matrix: np.ndarray, fi: Union[int, float]) -> Tuple[np.ndarray, np.ndarray, Union[int, float]]:
    """
    ================================================================================================
    Slajd 2
    Poszykiwanie kompletnego przydziału
    ================================================================================================
    
    Funkcja wyznacza zera niezależne oraz zera zależne.
    Zasada działania:

    1. Stworzenie listy list zawierającej informacje o pozycjach zer w każdym wierszu macierzy matrix
    (jeśli zera nie ma w danym wierszu lista zawiera -1) oraz zmiennej best_sol i best_sol_size odpowiednio
    przechowującej informacje o najlepszym rozwiązaniu i liczbie zer przez nie wyznaczonej

    2. Wyznaczenie kombinacji tych pozycji zer
        przykład:
            wejście [[1, 2, 3], [-1], [0, 2]]
            wyjście [(1, -1, 0), (1, -1, 2), (2, -1, 0), (2, -1, 2), (3, -1, 0), (3, -1, 2)]

    3. Wyznaczenie najlepszego dopasowania:
            1. Przejście po wszystkich elementach:
                jeśli wartości w elemencie się powtarzają (z wyjątkiem -1):
                        zamień kolejne powtórzenia wartości na -1
                jeśli nie:
                        jeśli liczba niezależnych zer z danego elementu > best_sol_size:
                            ustaw best_sol_size jako liczba niezależnych zer z danego elementu (tj. rozmiar elementu - liczba wartości "-1")
                            ustaw best_sol jako ten element
                        jeśli nie:
                            wybierz następny element
                jeśli best_sol_size wynosi tyle co rozmiar macierzy:
                    zakończ
                jeśli nie:
                    sprawdź kolejny element

    4. Utwórz macierz z zerami zależnymi i niezależnymi an podstawie best_sol
                    0 - wartość różna od zera w macierzy matrix
                    1 - zero niezależne w macierzy matrix
                    2 - zero zależne w macierzy matrix

    5. Jeśli best_sol_size wynosi tyle co rozmiar macierzy matrix:
            zwróć informacje o przydziale, koszt i zakończ algorytm
       Jeśli nie:
            wywołaj kolejny etap

    :param matrix: przekształcona macierz z etapu I
    :param fi: aktualny koszt
    :return: informacja o przydziale i koszt lub macierz z I etapu, macierz z zerami nie~/zależnymi, aktualny koszt
    """
    # etap 1
    lol: List[List[int]] = list()
    best_sol: Optional[Tuple[int]] = []
    best_sol_size: Optional[int] = 0
    size: int = matrix.shape[0]
    for row in matrix:
        lol.append([])
        for col in range(size):
            if row[col] == 0:
                lol[-1].append(col)
        if not len(lol[-1]):
            lol[-1].append(-1)

    # etap 2
    pred_sol: List[Tuple[int]] = list(product(*lol))

    # etap 3
    for elem in pred_sol:
        tmp = Counter(elem)
        tmp2 = False
        for i in range(size):
            if tmp[i] and tmp[i] > 1:
                tmp2 = True
                break
        if tmp2:
            elem = list(elem)
            for i in range(len(elem)):
                if elem[i] in elem[:i]:
                    elem[i] = -1
            elem = tuple(elem)
            tmp = Counter(elem)
        if (tmp[-1] and size - tmp[-1] > best_sol_size) or not tmp[-1]:
            best_sol_size = size - tmp[-1] if tmp[-1] else size
            best_sol = elem
        else:
            continue
        if best_sol_size == size:
            break

    # etap 4
    matrix_to_return: np.ndarray = np.zeros((size, size))
    for i in range(size):
        if lol[i][0] != -1:
            for el in lol[i]:
                matrix_to_return[i][el] = 2
    for i in range(size):
        if best_sol[i] != -1:
            matrix_to_return[i][best_sol[i]] = 1

    # etap 5
    if best_sol_size == size:
        fit: str = ""
        for i in range(size):
            fit += f"zadanie: {i} --> maszyna: {best_sol[i]}\n"
        print("\nMacierz zer niezależnych")
        print(matrix_to_return)
        print("Dopasowanie:\n", fit, "\nKoszt:\n", str(fi))

    else:
        print("\nMacierz zer niezależnych")
        print(matrix_to_return)
        return matrix, matrix_to_return, fi  # tu powinno być wywołanie kolejnej funkcji

## Krok 3 - Sprawdzenie liczby zer niezależnych

In [ ]:
#Zwraca minimalną listę wierszy i listę kolumn zawierających wszystkie zera w macierzy = pokrycie wierzchołkowe
def cross_zeros_off(A:np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    
    independent_zero = 1 #Oznaczenie zera niezależnego
    dependent_zero = 2 #Oznaczenie zera zależnego
    
    n = len(A) #Wymiar macierzy
    
    if n != len(A[0]): #Funkcja przyjmuje tylko macierze kwadratowe
        print("Macierz nie jest kwadratowa")
        return 0
    
    #True -> oznaczam wiersz lub kolumnę symbolem x
    B1 = np.zeros(n, bool) #Wektor zakreśleń wierszy 
    B2 = np.zeros(n, bool) #Wektor zakreśleń kolumn
    
    new = 1
    while new == 1: #Sprawdzenie czy w poprzedniej pętli dodano nowy element
        new = 0
        
        #-----Oznaczanie symbolem x każdy wiersz nie posiadający zera niezależnego-----
        for i in range(n):
            independent_zero_exist = False
            for j in range(n):
                if A[i][j] == independent_zero:
                    independent_zero_exist = True
            if independent_zero_exist == False:
                if B1[i] == False:
                    new = 1
                B1[i] = True
                

        #-----Oznaczanie symbolem x każdą kolumnę posiadajęcą zero zależne w oznaczonym wierszu-----
        for i in range(n):
            for j in range(n):
                if A[i][j] == dependent_zero and B1[i] == True:
                    if B2[j] == False:
                        new = 1
                    B2[j] = True


        #-----Oznaczanie symbolem x każdy wiersz mający zero niezależne w oznakowanej kolumnie-----
        for i in range(n):
            for j in range(n):
                if A[i][j] == independent_zero and B2[j] == True:
                    if B1[i] == False:
                        new = 1
                    B1[i] = True
    
    #-----Pokrycie wierzchołkowe = wszystkie nieoznakowane wiersze i oznakowane kolumny-----
    row = np.logical_not(B1)
    column = B2
    
    print("Oznaczone wiersze: ", row)
    print("Oznaczone kolumny: ", column)
    
    return row, column

#### Komentarz
Minimalna liczba lini wykreślających wszystkie zera jest równa aktualnej liczbie zer niezależnych. Algorytm nie bierzę pod uwagę jak wiele zer jest w danym wierszu, sprawdza tylko czy istnieją.

## Krok 4 - Próba powiększenia zbioru zer niezależnych